# SMA_Github data crawler

## !!! The code is written to use in the colab environment !!!

Please see the crawl content for your assigned repo name.

 If you want to use it on your local computer, please change the folder path and download the repo info of your computer first.

## Mount Drive to colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import packages and configs (TODO Here)

How to generate a github personal access tokens see: https://docs.github.com/en/enterprise-server@3.9/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens

Request limitation article see: https://docs.github.com/en/rest/using-the-rest-api/rate-limits-for-the-rest-api?apiVersion=2022-11-28

In [2]:
import json
import requests

# TODO: config your github access token and the parent project folder path here
access_token = ""
parent_folder = "/content/drive/MyDrive/SMA_Final Project" # put your folder name here

headers = {'Authorization': f'token {access_token}'}
max_pages = 1000 # config how much pages to crawl, 49 is recommended, it is close to 5000 request per hour

## Do not modify

In [3]:
def fetch_top_deep_learning_repos(topNum):
    # Search for repositories related to "Deep Learning" sorted by stars
    url = 'https://api.github.com/search/repositories'
    params = {'q': 'topic:deep-learning', 'sort': 'stars', 'order': 'desc', 'per_page': topNum}
    response = requests.get(url, headers=headers, params=params)
    repos_data = response.json()['items']
    return repos_data

def top_repo(top_num = 50):
    # Fetch top deep learning repositories
    top_repos = fetch_top_deep_learning_repos(topNum = top_num)

    all_repo_names = []
    for repo in top_repos:
        # Fetch repository infos
        repo_data = {
            'name': repo['name'],
            'id': repo['id'],
            'readme_url': repo['html_url'] + "/blob/master/README.md",
            'full_name': repo['full_name'],
            'stars': repo['stargazers_count'],
            'forks': repo['forks_count'],
            'description': repo['description'],
            'topics': repo['topics'],
            'star_url': repo['stargazers_url'],
            'contrib_url': repo['contributors_url'],
            'cur_star_page': 1,
            'cur_star_fin': 0,
            'cur_contrib_page': 1,
            'cur_contrib_fin': 0
        }
        all_repo_names.append(repo['name'])

        # Save all data to JSON file
        with open(parent_folder + '/crawler code/repo_info/' + repo['name'] + '.json', 'w') as json_file:
            json.dump(repo_data, json_file, indent=4)

    # Save all data to JSON file
    with open(parent_folder + '/crawler code/repo_info/00_repo_names.json', 'w') as json_file:
        json.dump(all_repo_names, json_file, indent=4)

In [6]:
def fetch_user_info(url, start_page, end_page, per_page=100):
    # Get the URL of the last page
    response = requests.get(url, headers=headers, params={'per_page': per_page})
    try:
      total_pages = int(response.links['last']['url'].split('page=')[-1])
    except:
      try:
        total_pages = int(response.links['next']['url'].split('page=')[-1])
      except:
        total_pages = 1

    print(total_pages)

    if end_page >= total_pages:
      end_page = total_pages

    print("Start Page:", start_page, ",End Page:", end_page)

    # Fetch pages within the specified range
    all_users_data = []
    while start_page <= end_page:
        if start_page % 100 == 0:
          print(start_page)
        response = requests.get(url, headers=headers, params={'per_page': per_page, 'page': start_page})
        if response.status_code == 200:
            users_data = response.json()
            all_users_data.extend(users_data)
            # Check if there's another page
            if 'next' in response.links:
                start_page += 1
            else:
                break
        else:
            # Handle other status codes if needed
            print(f"Error: {response.status_code}")
            return None

    # Determine state
    state = 0
    if start_page >= end_page:
      state = 1

    return all_users_data, state, end_page

# def fetch_url_count(url):
#     response = requests.get(url, headers=headers, params={'per_page': 1})
#     # print(response.headers['x-ratelimit-remaining']) # check whether you have request numbers left

#     if response.status_code == 200:
#         if 'last' in response.links:
#             # Get the URL of the last page
#             url = response.links['last']['url']
#             # Extract the page query parameter from the URL
#             total_pages = int(url.split('page=')[-1])
#             # Calculate the total count
#             url_count = (total_pages - 1) * 1 + len(response.json())
#         else:
#             # If there's no 'last' link, there's only one page
#             url_count = len(response.json())
#     else:
#         print("Error code:", response.status_code)
#         if (response.status_code != 403) and (response.status_code != 429):
#           return -1

#     return url_count

def starer_info(repo_path = ''):
      # Open the JSON file
      with open(repo_path, 'r') as json_file:
          # Load JSON data into a Python dictionary
          data = json.load(json_file)

      if data['cur_star_fin'] == 0:
        # Fetch starring user infos
        users_starring_repo, state, end_page = fetch_user_info(url = data['star_url'], start_page=data['cur_star_page'], end_page=data['cur_star_page'] + max_pages - 1)

        starred_users_data = []
        # count = 0
        for user in users_starring_repo:
            # count = count + 1
            user_data = {
                'username': user['login'],
                'id': user['id'],
                'followers_url': user['followers_url'],
                'repositories_url': user['repos_url'],
                'star_url': user['starred_url'],
                # 'followers_count': fetch_url_count(user['followers_url']),
                # 'repositories_count': fetch_url_count(user['repos_url']),
                # 'star_count': fetch_url_count(user['starred_url'].split('{')[0]),
            }
            starred_users_data.append(user_data)
            # if count % 100 == 0:
            #   print(count)

        with open(parent_folder + '/crawler code/starer_info/' + data['name'] + '.json', 'a') as json_file:
          # for i in starred_users_data:
          #   # Write new_data to the file
          #   json.dump(i, json_file, indent=4)
            json.dump(starred_users_data, json_file, indent=4)
            # json_file.write('\n')

        # Modify the specific key-value pair
        data['cur_star_page'] = end_page
        data['cur_star_fin'] = state
        # Write the modified data back to the file
        with open(repo_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)

        if state == 1:
          print("Repo starer finished!")

      else:
        print("Repo starer finished!")

def contrib_info(repo_path = ''):
      # Open the JSON file
      with open(repo_path, 'r') as json_file:
          # Load JSON data into a Python dictionary
          data = json.load(json_file)

      if data['cur_contrib_fin'] == 0:
        # Fetch contrib user infos
        users_contrib_repo, state, end_page = fetch_user_info(url = data['contrib_url'], start_page=data['cur_contrib_page'], end_page=data['cur_contrib_page'] + max_pages - 1)

        contrib_users_data = []
        # count = 0
        for user in users_contrib_repo:
            # count = count + 1
            user_data = {
                'username': user['login'],
                'id': user['id'],
                'followers_url': user['followers_url'],
                'repositories_url': user['repos_url'],
                'star_url': user['starred_url'],
                # 'followers_count': fetch_url_count(user['followers_url']),
                # 'repositories_count': fetch_url_count(user['repos_url']),
                # 'star_count': fetch_url_count(user['starred_url'].split('{')[0]),
                'contributions': user['contributions'],
            }
            contrib_users_data.append(user_data)
            # if count % 100 == 0:
            #   print(count)

        with open(parent_folder + '/crawler code/contrib_info/' + data['name'] + '.json', 'a') as json_file:
          # for i in contrib_users_data:
          #   # Write new_data to the file
          #   json.dump(i, json_file, indent=4)
            json.dump(contrib_users_data, json_file, indent=4)
            # json_file.write('\n')

        # Modify the specific key-value pair
        data['cur_contrib_page'] = end_page
        data['cur_contrib_fin'] = state
        # Write the modified data back to the file
        with open(repo_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)

        if state == 1:
          print("Repo contributor finished!")

      else:
          print("Repo contributor finished!")

## Do not modify end

## Finish both the starer_info and contrib_info crawler (TODO Here)

Do the two tasks one at a time, or you will exceed the rate limit. Remember to change the name of the repo. You may need to execute multiple times until you see the "Repo starer finished!" and "Repo contributor finished!" printed on the screen.

In [7]:
if __name__ == "__main__":
    # Run one of the lines below to get the data you want.
    # top_repo(top_num = 50) # get the top repo infos

    # TODO: pick one line
    repo_name = 'stable-diffusion-webui'
    starer_info(repo_path = parent_folder + '/crawler code/repo_info/' + repo_name + '.json') # get the starer user infos
    # contrib_info(repo_path = parent_folder + '/crawler code/repo_info/' + repo_name + '.json') # get the starer user infos

400
Start Page: 1 ,End Page: 400
100


KeyboardInterrupt: 

In [ ]:
# Check if you still have requests left
response = requests.get("https://api.github.com", headers=headers, params={'per_page': 1})
print(response.headers['x-ratelimit-remaining']) # check whether you have request numbers left

## Do not modify

In [ ]:
# # The code to reset the values in the repo_info data
# # Open the JSON file
# repo_path = '/content/drive/MyDrive/SMA_Final Project/crawler code/repo_info/' + 'stable-diffusion-webui' + '.json'
# with open(repo_path, 'r') as json_file:
#     # Load JSON data into a Python dictionary
#     data = json.load(json_file)

#     # Modify the specific key-value pair
#     data['cur_star_page'] = 1
#     data['cur_star_fin'] = 0
#     data['cur_contrib_page'] = 1
#     data['cur_contrib_fin'] = 0
#     # Write the modified data back to the file
#     with open(repo_path, 'w') as json_file:
#         json.dump(data, json_file, indent=4)

In [ ]:
# with open(parent_folder + '/crawler code/contrib_info/' + repo_name + '.json', 'r') as json_file:
#       data = json.load(json_file)
#       print(len(data))